In [2]:
def schedule_extract(username, password):
    # schedule extractor
    import re
    import time
    from selenium import webdriver
    from selenium.webdriver.common.by import By
    from bs4 import BeautifulSoup
    import pandas as pd
    
    driver = webdriver.Chrome()
    link = r"https://aisis.ateneo.edu/j_aisis/J_VMCS.do"
    
    driver.get(link)
    time.sleep(1)
    
    driver.find_element(By.NAME, 'userName').send_keys(username) #lmao i almost left my password in
    driver.find_element(By.NAME, 'password').send_keys(password)
    driver.find_element(By.NAME, "submit").click()
    time.sleep(1)
    
    driver.find_element(By.XPATH, '//a[contains(@href, "J_VMCS.do")]').click()
    time.sleep(1.5)
    
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    driver.quit()
    
    # find the table with the schedule
    all_tables = soup.find_all("table")
    schedule_table = None
    for table in all_tables:
        # check for presence of headers or specific keywords in table cells
        if any("Time" in cell.text.strip() for cell in table.find_all("td")[:2]):
            schedule_table = table
            break
    
    # extract data from the table and load it into dataframe
    rows = schedule_table.find_all("tr") # all rows are indicated by "tr" when printing the extracted soup
    data = [] # initializing
    headers = [header.text.strip() for header in rows[0].find_all("td")]
    # rows[0] accesses the first row, which for this soup object contains all the table headers 'Time', 'Mon, 'Tue', 'Wed', etc.
    # find_all("td") finds all the column elements inside this header row
    # header.text.strip() extracts the text, removes any whitespace, and stores the text it extracts as headers for the df
    
    for row in rows[1:]:
        # we skip the header row since it's already been skimmed over
        cols = row.find_all("td")
        # finding all the column elements across all rows barring the header row
        data.append([col.text.strip() for col in cols])
    
    sched_df = pd.DataFrame(data, columns=headers)
    
    # panda extractor
    def panda_schedule_extractor(df):
        schedules = []
        days = df.columns[1:]  # excluding the column for 'Time' to store the names of the days
    
        for day in days:
            current_class = None # initializing current_class and start_time as None
            start_time = None
            for index, row in df.iterrows(): # iterates over each row in the DataFrame
                time = row['Time'] # stores the current time range from column 'Time'
                class_name = row[day] # stores the class name for the current 'day'
    
                if class_name and class_name != current_class: # if class_name exists (not empty) and it's different from the current_class, the program reads it as a new class starting
                    if current_class: # if there's an ongoing class, it still occupies the current time slot
                        end_time = df.at[index-1, 'Time'].split('-')[1] #the end time of the current class is the end time of the previous time slot (hence index-1) 
                        schedules.append(f"{current_class} {current_day} {start_time}-{end_time}") #all this is appended to the schedule string here
                    current_class = class_name # the current_class is updated to the new name of the new class
                    current_day = day # current_day is updated to the day of the new class
                    start_time = time.split('-')[0] # start time is set to the start time of the current time_slot
                elif class_name != current_class: # this elif-if checks if an ongoing class is about to end
                    if current_class: # if there's an ongoing class and class_name 
                        end_time = df.at[index-1, 'Time'].split('-')[1] 
                        schedules.append(f"{current_class} {current_day} {start_time}-{end_time}")
                    current_class = None
    
            # Handle the last ongoing class of the day
            if current_class:
                end_time = time.split('-')[1] # end time of the last ongoing class is the end time of the current time slot
                schedules.append(f"{current_class} {current_day} {start_time}-{end_time}")
    
        return schedules
    
    # Extract schedules
    class_schedules = panda_schedule_extractor(sched_df)
    
    #print(class_schedules)
    
    final_sched_df = pd.DataFrame(columns = ['Subject', 'Location', 'Day', 'Start Time', 'End Time'])
    
    
    
    def parse_schedule(schedule):
        # Split by spaces and parentheses
        parts = item.split()
        
        # Extract subject and location
        subject = ' '.join(parts[0:2])
        location = ' '.join(parts[2:-2])
        
        # Extract day and time
        day = parts[-2]
        start_time_str, end_time_str = parts[-1].split('-')
        
        # Format times
        start_time = f"{start_time_str[:-2]}:{start_time_str[-2:]}"
        end_time = f"{end_time_str[:-2]}:{end_time_str[-2:]}"
    
        temp_list = [subject, location, day, start_time, end_time]
    
        return temp_list
    
    for item in class_schedules:
        final_sched_df.loc[len(final_sched_df.index)] = parse_schedule(item)
    
    def section_specify(subject):
        section = subject[-1] 
        final_subject = subject[0:-1]
        final_subject += f' Section {section}'
        return final_subject
    
    final_sched_df['Subject'] = final_sched_df['Subject'].apply(section_specify)
    return final_sched_df

,Subject,Location,Day,Start Time,End Time
0,ArtAp 10 Section J,INNOVATION 202 (FULLY ONSITE),Mon,11:30,15:00
1,ITMGT 25.03 Section N,SEC-A123 (FULLY ONSITE),Tue,11:30,15:00
2,ArtAp 10 Section J,INNOVATION 202 (FULLY ONSITE),Thur,12:00,16:00
3,ITMGT 25.03 Section N,SEC-A123 (FULLY ONSITE),Fri,12:00,16:00


In [6]:
def create_events():
    import os.path  # to handle file paths
    from datetime import datetime as dt # for handling dates and times
    import pytz  # Import the pytz library for time zone management
    
    # Import necessary modules for Google authentication and API access
    from google.auth.transport.requests import Request  # For handling HTTP requests during authentication
    from google.oauth2.credentials import Credentials  # For managing OAuth 2.0 credentials
    from google_auth_oauthlib.flow import InstalledAppFlow  # For handling OAuth 2.0 flow for installed apps
    from googleapiclient.discovery import build  # For creating a service object to access the Google API
    from googleapiclient.errors import HttpError  # For handling HTTP errors from the API
    
    from datetime import datetime
    
    def parse_date(date_str):
        # Define the year you want to use
        year = datetime.now().year
    
        # Parse the date string "JULY 13" to "YYYY-MM-DD"
        date_obj = datetime.strptime(f"{year} {date_str}", "%Y %B %d")
        return date_obj.date().isoformat()
        
    Scopes = ['https://www.googleapis.com/auth/calendar']
    
    
    creds = None
    if os.path.exists('token.json'):
        creds = Credentials.from_authorized_user_file('token.json')
    
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file('credentials.json', Scopes)
            creds = flow.run_local_server(port = 0)
    
        with open('token.json','w') as  token:
            token.write(creds.to_json())
        
    service = build("calendar",'v3', credentials = creds)
    manila_tz = pytz.timezone('Asia/Manila')
        
    googleSheetId = '1m8badixRPCpTt-dKThNeqW98PK4naJpHsAoUKA_0Uro'
    workSheetName = 'Syllabus_Dates'
    URL = 'https://docs.google.com/spreadsheets/d/{0}/gviz/tq?tqx=out:csv&sheet={1}'.format(googleSheetId, workSheetName)
    date_df = pd.read_csv(URL)
        
    date_df['Summary'] = date_df.apply(lambda x: x.Subject + ' - ' + x.Summary, axis = 1)
        
    del date_df['Subject']
    date_df['Description'] = date_df['Description'].apply(lambda x: '' if pd.isna(x) else x)
    date_df['Color ID'] = date_df['Color ID'].apply(lambda x: 0 if pd.isna(x) else int(x.split(' - ')[0]))
        
    for index, row in date_df.iterrows():  
        event = {
                    'summary': row['Summary'],
                    'location': row['Location'],
                    'description': row['Description'],
                    'colorId': row['Color ID'],
                    'start': {
                        'date': parse_date(row['Date']),
                        'timeZone': 'Asia/Manila'
                    },
                    'end': {
                        'date': parse_date(row['Date']),
                        'timeZone': 'Asia/Manila'
                    },
                    'recurrence': ['RRULE:FREQ=DAILY;COUNT=1'],
                }
        event = service.events().insert(calendarId = 'primary', body = event).execute()
    
        print(f"Event {index + 1} of {len(date_df)} created {event.get('htmlLink')}")


    '===================================================='
    
    googleSheetId2 = '1HIzSiip1TT7wwGRrLnxLwEnqickJrbc-EunKEmY-z_k'
    workSheetName2 = 'Weekly_Schedule'
    URL2 = 'https://docs.google.com/spreadsheets/d/{0}/gviz/tq?tqx=out:csv&sheet={1}'.format(googleSheetId2, workSheetName2)
    schedule_df = pd.read_csv(URL2)
    
    schedule_df['Color ID'] = schedule_df['Color ID'].apply(lambda x: 0 if pd.isna(x) else int(x.split(' - ')[0]))
    
    schedule_df['Day'] = schedule_df['Day'].apply(lambda x: x[0:2].upper())
    schedule_df
    
    def parse_datetime(year, date_str, time_str):
        # Combine the provided date string with the time string to create a datetime object
        datetime_obj = dt.strptime(f"{year} {date_str} {time_str}", "%Y %B %d %H:%M")
        return datetime_obj.isoformat()
    
    def create_weekly_event(row, current_year):
        start_datetime = parse_datetime(current_year, row['Start of Classes'], row['Start Time'])
        end_datetime = parse_datetime(current_year, row['Start of Classes'], row['End Time'])
        
        # Convert end_date_str to the UNTIL format
        end_date_obj = dt.strptime(row['End of Classes'], "%B %d")
        until_date = end_date_obj.replace(year=current_year).strftime("%Y%m%dT235959Z")
        
        
        byday = row['Day']
        
        event = {
            'summary': row['Subject'],
            'location': row['Location'],
            'description': row['Description'],
            'colorId': row['Color ID'],  
            'start': {
                'dateTime': start_datetime,
                'timeZone': 'Asia/Manila'
            },
            'end': {
                'dateTime': end_datetime,
                'timeZone': 'Asia/Manila'
            },
            'recurrence': [f'RRULE:FREQ=WEEKLY;BYDAY={byday};UNTIL={until_date}'],
        }
        return event
    
    current_year = dt.now().year
    
    for index, row in schedule_df.iterrows():
        event = create_weekly_event(row, current_year)

        event = service.events().insert(calendarId = 'primary', body = event).execute()
        print(f"Class {index + 1} of {len(schedule_df)} uploaded!")

    print('Everything is finished uploading! Please check out your calendar!')

create_events()

Event 1 of 7 created https://www.google.com/calendar/event?eid=Y2U0NmNnYWY3djBiODBqcXUzdHJldWxraWNfMjAyNDA2Mjcgc3B5dWl0bWd0QG0
Event 2 of 7 created https://www.google.com/calendar/event?eid=anU0Y29udDl1ajcwOWZqbWI4dHM3OGk0bmtfMjAyNDA3MDQgc3B5dWl0bWd0QG0
Event 3 of 7 created https://www.google.com/calendar/event?eid=OWk2Z3E2cjFucGZ0NG4waTgza3Y2Z2cwOTBfMjAyNDA3MTEgc3B5dWl0bWd0QG0
Event 4 of 7 created https://www.google.com/calendar/event?eid=NWNodGJwM200aWZzcDF0ODlkdDZhMDliZThfMjAyNDA3MTcgc3B5dWl0bWd0QG0
Event 5 of 7 created https://www.google.com/calendar/event?eid=bjRpZG9vMGg3N2p0MnV2cWcyODkyZ3MyM2tfMjAyNDA2MTkgc3B5dWl0bWd0QG0
Event 6 of 7 created https://www.google.com/calendar/event?eid=ZjhzdTUzcGtuMzlzcmowcm5sM3RiOHZvNjRfMjAyNDA2MjYgc3B5dWl0bWd0QG0
Event 7 of 7 created https://www.google.com/calendar/event?eid=bnZqZHFwZjk5NTUzN2NqdDRlZjFpMmY0cmNfMjAyNDA3MDEgc3B5dWl0bWd0QG0
Class 1 of 4 uploaded!
Class 2 of 4 uploaded!
Class 3 of 4 uploaded!
Class 4 of 4 uploaded!


,Color ID,Subject,Location,Day,Start Time,End Time,Start of Classes,End of Classes,Description
0,1,ArtAp 10 Section J,INNOVATION 202 (FULLY ONSITE),MO,11:30,15:00,June 5,July 20,1
1,2,ITMGT 25.03 Section N,SEC-A123 (FULLY ONSITE),TU,11:30,15:00,June 5,July 20,2
2,1,ArtAp 10 Section J,INNOVATION 202 (FULLY ONSITE),TH,12:00,16:00,June 5,July 20,3
3,2,ITMGT 25.03 Section N,SEC-A123 (FULLY ONSITE),FR,12:00,16:00,June 5,July 20,4


In [73]:
from datetime import datetime

def parse_datetime(year, date_str, time_str):
    # Combine the provided date string with the time string to create a datetime object
    datetime_obj = datetime.strptime(f"{year} {date_str} {time_str}", "%Y %B %d %H:%M")
    return datetime_obj.isoformat()

def create_weekly_event(row, current_year):
    start_datetime = parse_datetime(current_year, row['Start of Classes'], row['Start Time'])
    end_datetime = parse_datetime(current_year, row['Start of Classes'], row['End Time'])
    
    # Convert end_date_str to the UNTIL format
    end_date_obj = datetime.strptime(row['End of Classes'], "%B %d")
    until_date = end_date_obj.replace(year=current_year).strftime("%Y%m%dT235959Z")
    
    
    byday = row['Day']
    
    event = {
        'summary': row['Subject'],
        'location': row['Location'],
        'description': row['Description'],
        'colorId': row['Color ID'],  
        'start': {
            'dateTime': start_datetime,
            'timeZone': 'Asia/Manila'
        },
        'end': {
            'dateTime': end_datetime,
            'timeZone': 'Asia/Manila'
        },
        'recurrence': [f'RRULE:FREQ=WEEKLY;BYDAY={byday};UNTIL={until_date}'],
    }
    return event

current_year = datetime.now().year

for index, row in schedule_df.iterrows():
    event = create_weekly_event(row, current_year)
    print(event)

{'summary': 'ArtAp 10 Section J', 'location': 'INNOVATION 202 (FULLY ONSITE)', 'description': 1, 'colorId': 1, 'start': {'dateTime': '2024-06-05T11:30:00', 'timeZone': 'Asia/Manila'}, 'end': {'dateTime': '2024-06-05T15:00:00', 'timeZone': 'Asia/Manila'}, 'recurrence': ['RRULE:FREQ=WEEKLY;BYDAY=MO;UNTIL=20240720T235959Z']}
{'summary': 'ITMGT 25.03 Section N', 'location': 'SEC-A123 (FULLY ONSITE)', 'description': 2, 'colorId': 2, 'start': {'dateTime': '2024-06-05T11:30:00', 'timeZone': 'Asia/Manila'}, 'end': {'dateTime': '2024-06-05T15:00:00', 'timeZone': 'Asia/Manila'}, 'recurrence': ['RRULE:FREQ=WEEKLY;BYDAY=TU;UNTIL=20240720T235959Z']}
{'summary': 'ArtAp 10 Section J', 'location': 'INNOVATION 202 (FULLY ONSITE)', 'description': 3, 'colorId': 1, 'start': {'dateTime': '2024-06-05T12:00:00', 'timeZone': 'Asia/Manila'}, 'end': {'dateTime': '2024-06-05T16:00:00', 'timeZone': 'Asia/Manila'}, 'recurrence': ['RRULE:FREQ=WEEKLY;BYDAY=TH;UNTIL=20240720T235959Z']}
{'summary': 'ITMGT 25.03 Sectio